# Trabalho Prático 0

## Introdução

A resolução deste trabalho prático tem como objetivo servir de iniciação à componente prática da unidade curricular de Estruturas criptográficas, onde se pretende:
   - Instalar as ferramentas computacionais necessárias para a realização dos trabalhos práticos.
   - Demonstrar pequenas aplicações implementadas em **Python** e em **SageMath**.

A aplicação em **Python** deve ser implementada de tal forma que permita a comunicação entre um emissor e um recetor, com as seguintes caracteristicas:
   - Criptograma e metadados devem ser autenticados.
   - Utilizar uma _stream cipher_.
   - Autenticar previamente a chave.
    
A aplicação em **SageMath** deve ser implementada de tal forma que:
   - Crie 4 corpos finitos primos.
   - Crie um _plot_ de uma função em cada um dos corpos finitos primos.
   - Teste, por amostragem, uma proposição.

## Aplicação Python

### Imports

Esta secção executa os _imports_ de **Python** que contêm as funções necessárias para executar a aplicação desenvolvida.

In [12]:
import os
from getpass import getpass
from multiprocessing import Process,Pipe
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

### Definição da classe de multiprocessamento

Esta secção tem o propósito de definir a classe de multiprocessamento, que permite uma comunicação bidireccional com o _Emitter_ e o _Receiver_.

In [5]:
class BiConnection(object):
    def __init__(self,right,left):
        left_side,right_side = Pipe()
        self.timeout = None
        self.left_process = Process(target=left,args=(left_side,))
        self.right_process = Process(target=right,args=(right_side,))
        self.left = lambda : left(left_side)
        self.right = lambda : right(right_side)
    
    def manual(self): # Execução manual apenas devido ao facto de a password ter que ser lida em ambos os lados do Pipe
        self.left()
        self.right()
    

### Definição do Emissor e do Recetor

Esta secção tem o propósito de definir o emissor e o recetor nesta comunicação bidireccional, bem como as acções que cada um deve executar nesta aplicação.

In [22]:
def Emissor(connection):
    con_salt = os.urandom(16)
    passwd = bytes(getpass('Password do emissor: '),'utf-8')
    text_to_send = os.urandom(1024)
    try:
        derivation = PBKDF2HMAC(
                algorithm = hashes.SHA256(),
                length = 64,
                salt = con_salt,
                iterations = 100000,
                backend = default_backend()
        )
        full_key = derivation.derive(passwd)
        cript_key = full_key[:32]
        mac_key = full_key[32:]
        # Utilizar o AES com um dos modos que simula a stream cipher.
        nonce = os.urandom(16)
        cipher = Cipher(algorithm = algorithms.AES(cript_key),mode= modes.CTR(nonce),backend = default_backend())
        cryptogram = cipher.encryptor().update(text_to_send)
        message_to_authenticate = cryptogram + nonce
        hash_msg = hmac.HMAC(mac_key,hashes.SHA256(),default_backend()).update(message_to_authenticate)
        obj = {'cryptogram': cryptogram, 'mac_code': hash_msg, 'salt': con_salt, 'nonce': nonce}
        connection.send(obj)
    except Exception as e:
        print(e)
